# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [56]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [57]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [58]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @JoaoAnd93050623

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [59]:
#Produto escolhido:
produto = 'SPTrans'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [64]:
#Cria um objeto para a captura
api = tweepy.API(auth)


#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode='extended').items():  
    if (not msg.retweeted) and ('RT' not in msg.full_text) and msg.author.screen_name != "sptrans": 
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [65]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

C:\Users\crazy\AppData\Local\Continuum\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:912: UserWarning: Ignoring URL 'https://t.co/fjla1rzemh%20%20%20

olá%20@sptrans%20precisamos%20de%20mais%20transporte%20público%20para%20os%20cidadãos%20e%20não%20a%20redução%20das%20frotas,%20ainda%20mais%20de%20linhas%20tão%20sobrecarregadas.%20como%20a%20diminuição%20de%20saídas%20do%20637a-26.

' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))


___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [123]:
SPTrans = pd.read_excel('SPTrans.xlsx')
SPTrans_I = SPTrans[SPTrans.Classificacao == "I"]
SPTrans_R = SPTrans[SPTrans.Classificacao == "R"]
SPTrans_R = SPTrans_R.reset_index()
SPTrans_R = SPTrans_R.drop(columns = ["index"])
SPTrans_I = SPTrans_I.reset_index()
SPTrans_I = SPTrans_I.drop(columns = ["index"])
Irrelevantes_raw = []
Relevantes_raw = []

In [118]:
SPTrans_R

,Treinamento,Classificacao
0,@joelspinelli @sptrans boa sorte para encontra...,R
1,@sptrans não vou perder o meu tempo! vocês só ...,R
2,"@sptrans essas mudanças no vt são ridículas, n...",R
3,alô @sptrans problemas no site para cadastro d...,R
4,#sptrans o que acontece dessa vez? ou outra ve...,R
5,@sptrans qdo coloco a data aparece essa mensag...,R
6,@sptrans\n \n3 carros da linha 3459-10 passara...,R
7,paralisação \napós conversar com alguns motori...,R
8,eu tento frequentar as aulas mas ai a sptrans ...,R
9,parabéns aos responsáveis que cobram muito e n...,R


In [124]:
for i in range(0, len(SPTrans_R.Treinamento)):
    Relevantes_raw.append(SPTrans_R.Treinamento[i])
for i in range(0, len(SPTrans_I.Treinamento)):
    Irrelevantes_raw.append(SPTrans_I.Treinamento[i])

In [149]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text = ' '.join(word for word in text.split() if not word.startswith('https'))
    text = ' '.join(word for word in text.split() if not word.startswith('@'))
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed  

In [152]:
Relevantes_raw_txt = ', '.join(Relevantes_raw)
Irrelevantes_raw_txt = ','.join(str(v) for v in Irrelevantes_raw)
Relevantes = cleanup(Relevantes_raw_txt.lower())
Irrelevantes = cleanup(Relevantes_raw_txt.lower())

In [157]:
Serie_Relevantes  = pd.Series(Relevantes.split())
Tabela_Relevantes = Serie_Relevantes.value_counts()
Serie_Irrelevantes  = pd.Series(Irrelevantes.split())
Tabela_Irrelevantes = Serie_Irrelevantes.value_counts()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**